## Design a recursion function 

### Interpret Recursion

Consider we want to solve a problem $F(X)$, we can decompose it into several small problems: $x_0 \in X, x_1 \in X, ....$

Base case: 
- **The solution of the minimum scale problem is the base case** of the recursive function, 
- through which the <u>pop operation</u> of the stack is realized until the stack is empty, and the problem is solved.
- In base case, the problem can be solved directly, and base case is usually taken as the jump out condition of recursion.

Recursion relationship: 
- **The relationship between the result of the problem and results of its sub-problems**
- Writing out the recursive relationship is equivalent to implementing the <u>push operation</u> of the stack. 

> **Recursion is a programming technique, not an algorithm**. *Divide and conquer, dynamic programming, and backtracking* are algorithms. The relationship between the two is that the most commonly used implementation of these three algorithms is recursion.<br/>
In both the divide and conquer algorithm and the dynamic programming algorithm, it is necessary to disassemble the large problem into sub-problems.

### **wrapper funtion**


A wrapper function is a function that is directly called but does not recurse itself, instead calling a separate *auxiliary function* which actually does the recursion.

Wrapper functions can be used to <u>validate parameters (so the recursive function can skip these)</u>, perform initialization (allocate memory, initialize variables), particularly for auxiliary variables such as "level of recursion" or <u>partial computations for memoization</u>, and handle exceptions and errors.

> Wrapper functions can reduce the number of parameters in the recursive call, which reduces the stack usage, as all parameters are pushed on the stack for each call, and if there are many parameters and many layers of recursion, there could be a stack overflow. Any parameters that do no change for the recursive call can be left in an enclosing lexical scope or passed altogether as a reference to a structure.

```c
\\ Ordinary
int fac1(int n) {
   if (n <= 0)
      return 1;
   else
      return fac1(n-1)*n;
}
```

```c
\\ Short-circuit recursion 
static int fac2(int n) {
   // assert(n >= 2);
   if (n == 2)
      return 2;
   else
      return fac2(n-1)*n;
}
int fac2wrapper(int n) {
   if (n <= 1)
      return 1;
   else
      return fac2(n);
}
```

**Short-circuiting the base case**:
Short-circuiting the base case, also known as **arm's-length recursion**, consists of checking the base case before making a recursive call – *i.e.*, checking if the next call will be the base case, instead of calling and then checking for the base case.

For example, in the factorial function, <u>properly the base case is 0! = 1, while immediately returning 1 for 1! is a short circuit</u>, and may miss 0; this can be mitigated by a wrapper function. 

- Short-circuiting is particularly done for efficiency reasons, to avoid the overhead of a function call that immediately returns.
- Short-circuiting is primarily a concern when many base cases are encountered, such as `Null` pointers in a tree, which can be linear in the number of function calls, hence significant savings for $O(n)$ algorithms; this is illustrated below for a depth-first search. 
- Short-circuiting on a tree corresponds to considering <u>a leaf (non-empty node with no children) as the base case</u>, rather than considering an empty node as the base case. If there is only a single base case, such as in computing the factorial, short-circuiting provides only O(1) savings. 


#### DFS

The standard recursive algorithm for a DFS is:

- base case: If current node is Null, return false
- recursive step: otherwise, check value of current node, return true if match, otherwise recurse on children

In short-circuiting, this is instead:

- check value of current node, return true if match,
- otherwise, on children, if not Null, then recurse.


In the case of a perfect binary tree of height $h$, there are $2^{h+1}−1$ nodes and $2^{h+1}$ `Null` pointers as children ($2$ for each of the $2^h$ leaves), so short-circuiting cuts the number of function calls in half in the worst case. 

```c
bool tree_contains(struct node *tree_node, int i) {
    if (tree_node == NULL)
        return false;  // base case
    else if (tree_node->data == i)
        return true;
    else
        return tree_contains(tree_node->left, i) ||
               tree_contains(tree_node->right, i);
}

```

```c
// Wrapper function to handle empty tree
bool tree_contains(struct node *tree_node, int i) {
    if (tree_node == NULL)
        return false;  // empty tree
    else
        return tree_contains_do(tree_node, i);  // call auxiliary function
}

// Assumes tree_node != NULL
bool tree_contains_do(struct node *tree_node, int i) {
    if (tree_node->data == i)
        return true;  // found
    else  // recurse
        return (tree_node->left  && tree_contains_do(tree_node->left,  i)) ||
               (tree_node->right && tree_contains_do(tree_node->right, i));
}
```

### **How to write a recursive function**

1. Find the basic case;
2. Find the recurrence relationship.

There are 2 paradigms we can follow: **from bottom to up, from up to bottom**.

#### From bottom to up

**Main idea**: first, call the recursive function itself; then, perform calculations based on the return value.

**Example**:
```java
/** 
 * Simulation process：
 * 5 + sum(4)
 * 5 + (4 + sum(3)
 * 5 + 4 + (3 + sum(2))
 * 5 + 4 + 3 + (2 + sum(1))
 * ------------------> jump out until sum(1) = 1
 * 5 + 4 + 3 + (2 + 1)
 * 5 + 4 + (3 + 3)
 * 5 + (4 + 6)
 * (5 + 10)
 * 15
 * from up to bottom: calculated as 1 + 2 + 3 + 4 + 5 
 */
public int sum(int n) {
    if (n < 2) return n;       // ① base case
    int childSum = sum(n - 1); // ② find recursive relationship
    return n + childSum;       // ③ calculate based on the return result
}
```

**Bottom-up paradigm**:
1. Find the base case and return the result of the base case when jumping out;
2. **Modify the parameters** of the recursive function;
3. Call the recursive function and get **intermediate variables**;
4. Use the result of the recursive function to calculate the final result with the current parameters;
5. Return the final result.

```python
def func(params):
    if (base case): return res of base case
    current params = modify(params)
    inter var = func(current params)
    final res = calculate(inter var, current params)
    return final res
```

#### From up to bottom

**Main idea**: calculate the intermediate variables based on current parameters, and then modify parameters, **pass these intermediate variables to the recursive function with modified parameters**, at last, **return the recursive function itself**.  

**Example**:
```java

/**
 * 模拟程序执行过程：
 * sum(5, 0)
 * sum(4, 5)
 * sum(3, 9)
 * sum(2, 12)
 * sum(1, 14)
 * 15
 * from up to bottom: calculated as  5 + 4 + 3 + 2 + 1 
 */
public int sum2(int n, int sum) {
    if (n < 2) return 1 + sum;
    sum += n;
    return sum2(n - 1, sum);
}
```

**Up-bottom paradigm**:
1. find the base case, and return the result of the base case (`1`) and calculation results of the intermediate variables (`sum`)
2. recalculate new intermediate variables based on current parameters.
3. Modify the parameters of the recursive function;
4. **Return the recursive function, with new intermediate variables and modified parameters as arguments.**

```python
def func(inter var, params):
    if (base case): return res of base case + inter var
    new inter var = calculate(inter var, params)
    new params = modify(params)
    return func(new inter var, new params)
```


#### Difference between Botton-up and Up-bottom

![](https://pic.leetcode-cn.com/367f77a847ab554d6c00a396b91d79a0c1effcbbc89d17d773e91a34345304e5-digui.jpg)

## Case Study

---
###  Recursive multiplication

**Description**: Write a recursive function to multiply two positive integers without using the * operator. Plus, minus, shifts are fine, but be stingy. 

**Example**:
```
Input  : A = 1, B = 10 
  Output  : 10 
```


#### Plus


**Analysis**:
- Base case:
   when `A = 0` (or `B = 0`), return `B + sum` (or `A + sum`);
- Recursive relationship:
   `sum += B`, if `A` is taken as the counter; otherwise, `sum += A`

> **NB**: solve this problem by adding one by on, which may exceed the limit of the maximum recursion number, such as `A = 343525325` and `B = 1`.

In [9]:
def multiply(A: int, B: int) -> int:
    if A == 0 or B == 0: return 0
    def multi_helper(A: int, B: int, sum: int) -> int:
        if A == 1: return B + sum
        sum += B
        return multi_helper(A - 1, B, sum)
    if A > B:
        A, B = B, A
    return multi_helper(A, B, 0)

A, B = 3, 4
multiply(A, B)

12

#### Bitwise shift

**Analysis**:
- Base case: if `A = 1`, return `B`
- Recursive relationship: consider `A >> 1` when `A > 1`, there are 2 possible situation:
  - if `A` is an odd, , `A * B` = `(A >> 1) * (B << 1) + B`
  - if `A` is an even, `A * B` = `(A >> 1) * (B << 1)`

In [30]:
def multiply(A: int, B: int) -> int:
    if A == 0 or B == 0: return 0
    def multi_helper(A: int, B: int, sum: int) -> int:
        if A == 1: return sum + B
        if A % 2: sum += B
        A >>= 1
        B <<= 1
        return multi_helper(A, B)
    if A > B: A, B = B, A
    return multi_helper(A, B, 0)

A, B = 14, 73807517
multiply(A, B)

7 147615034
3 442845102
1 1328535306


1328535306

---
### Minimum depth of tree

Example:
1. - Input:  root = `[3,9,20,null,null,15,7]` 
   - output:  `2`
2. - Input: root = `[2,null,3,null,4,null,5,null,6]`
   - output: `5`

- Base case: if `null`, return `0`; if leaf nodes, return `1`;
- Recursion Relationship: 
  1. if both left child and right child, depth = smaller depth + 1;
  2. if only left/right child, depth = depth of left/right child + 1;
  3. if neither left child nor right child, depth = 1;

In [1]:
from binarytree import build, build2

In [2]:
arr =  [3, 9, 20, None, None, 15, 7]
arr2 = [2, None, 3, None, 4, None, 5, None, 6]
root = build(arr)
root2 = build2(arr2)

#### DFS Solution

In [3]:
def min_depth(root):
    # empty tree/subtree
    if root is None: return 0  
    ld = min_depth(root.left)
    rd = min_depth(root.right)
    # has only left/right child, or probably is a leaf node
    if root.left is None or root.right is None:
        return ld + rd + 1  # ld or rd must be 0 (both 0 if a leaf node)
    # both left and right child
    return min(ld, rd) + 1 

print(min_depth(root))
print(min_depth(root2))

2
5


In [4]:
def minDepth(root) -> int:
    if not root: return 0
    elif not root.left: return minDepth(root.right) + 1
    elif not root.right: return minDepth(root.left) + 1
    else: return min(minDepth(root.left), minDepth(root.right)) + 1

print(minDepth(root))
print(minDepth(root2))

2
5


- **Time complexity**: $O(N)$
- **Space complexity**: $O(H)$, where $H$ is the height of the tree.
  - Worst case: the tree is chain-like, $H = N$, *i.e.*, $O(N)$;
  - Average case: $H = \log N$, *i.e.*, $O(\log N)$.

#### BFS Solution

In [5]:
def min_depth_bfs(root):
    if root is None: return 0
    queue = [(root, 1)]
    while queue:
        node, depth = queue.pop(0)
        if not node.left and not node.right:
            return depth
        if node.left:
            queue.append((node.left, depth + 1))
        if node.right:
            queue.append((node.right, depth + 1))
            
min_depth_bfs(root), min_depth_bfs(root2)

(2, 5)

In [6]:
def min_depth_bfs(root):
    if not root: return 0
    queue = [root]
    depth = 1
    while queue:
        num = len(queue)
        for _ in range(num):
            node = queue.pop(0)
            if not node.left and not node.right:
                return depth
            if node.left:
                queue.append(node.left)
            if node.right:
                queue.append(node.right) 
        depth += 1
        
min_depth_bfs(root), min_depth_bfs(root2)

(2, 5)

- **Time complexity**: $O(N)$
- **Space complexity**: $O(N)$, the space complexity depends on the size of the queue.

---
### Path Sum

**Requirements**:

gives you the root node of the binary tree root and an integer representing the target sum `targetSum`. Determine whether there is a path from the root node to the leaf node in the tree , and the sum of all node values ​​on this path is equal to the target sum `targetSum`. If exists, return true; otherwise, return `false`.

![](https://assets.leetcode.com/uploads/2021/01/18/pathsum1.jpg)
```
  input:  root = [5,4,8,11,null,13,4,7,2,null,null,null,1], targetSum = 22
  output:  true 
  Explanation:  The root node to leaf node path equal to the target sum is shown in the figure above. 
 
```


**Analysis**:
- Basic case: if the node is `None`, return `False`; if the node is a leaf node, judge whether the node value equals to the expected value. 
- Recursion relationship: 
  1. the path sum of the **left** subtree == `sum - root.val`
  2. the path sum of the **right** subtree == `sum - root.val`

In [7]:
arr = [5,4,8,11,None,13,4,7,2,None,None,None,1]
root = build(arr)
targetSum = 22 
print(root)


         5_____
        /      \
    ___4     ___8
   /        /    \
  11       13     4
 /  \        \
7    2        1



#### DFS Solution

In [8]:
def pathSum(root, targetSum):
    if not root: return False
    if not root.left and not root.right: 
        return (root.value == targetSum)
    return pathSum(root.left, targetSum-root.value) \
        or pathSum(root.right, targetSum-root.value) 
        
pathSum(root, targetSum)

True

#### BFS Solution

In [9]:
def pathSum(root, targetSum):
    if not root: return False
    queue = [(root, root.value)]
    while queue:
        node, value = queue.pop(0)
        if not node.left and not node.right:
            if targetSum == value:
                return True
            continue
        if node.left:
            queue.append((node.left, value + node.left.value))
        if node.right:
            queue.append((node.right, value + node.right.value))

pathSum(root, targetSum)

True

---
### Symmetric binary tree

**Requirements**:

```
Input:  root = [1,2,2,3,4,4,3] 
  output:  true 
 ```

```
Input:  root = [1,2,2,null,3,null,3] 
  output:  false 
 ```

**Analysis**:
- Basic case:
- Recursion relationship:

In [10]:
arr1 = [1,2,2,3,4,4,3]
root1 = build(arr1)
arr2 = [1,2,2,None,3,None,3]
root2 = build(arr2)

#### Recursion / DFS

In [11]:
def is_symmetric(root):
    if root is None: return True
    def symmetric_helper(left, right):
        if left is None and right is None:
            return True
        elif left is None or right is None:
            return False
        return left.value == right.value and \
            symmetric_helper(left.right, right.left) and \
            symmetric_helper(left.left, right.right)
    return symmetric_helper(root.left, root.right)

is_symmetric(root1), is_symmetric(root2)

(True, False)

- **Time complexity**: $O(N)$
- **Space complexity**: $O(H)$, where $H$ is the height of the tree.
  - Worst case: the tree is chain-like, $H = N$, *i.e.*, $O(N)$;
  - Average case: $H = \log N$, *i.e.*, $O(\log N)$.

#### Iterative / Queue

In [12]:
def is_symmetric_queue(root):
    if not root or not (root.left or root.right): return True
    queue = [root.left, root.right]
    while queue:
        left = queue.pop(0)
        right = queue.pop(0)
        if not (left or right):
            continue
        if not (left and right):
            return False
        if left.value != right.value:
            return False
        queue.append(left.left)
        queue.append(right.right)
        queue.append(left.right)
        queue.append(right.left)
    return True

is_symmetric_queue(root1), is_symmetric_queue(root2)

(True, False)

- **Time complexity**: $O(N)$
- **Space complexity**: $O(N)$, the space complexity depends on the size of the queue.

---
### Full permutation

**Requirements**: Given an array with no duplicate numbers `nums`, return all possible full permutations of it. You can return answers in any order. 

**Examples**:
```
Input:  nums = [1,2,3] 
  Output:  [[1,2,3],[1,3,2],[2,1,3],[2,3,1],[3,1,2],[3,2,1] ] 
 ```

 ```
 Input:  nums = [0,1] 
  Output:  [[0,1],[1,0]] 
  ```

**Analysis**:
- Basic case:
- Recursion relationship:

#### Recursion

In [13]:
arr = [1, 2, 3]

In [15]:
from typing import List

def permute(nums: List[int]) -> List[List[int]]:
    if len(nums) == 1: return [nums]
    pointer = nums[0]
    combs = permute(nums[1:])
    size = len(combs)
    for _ in range(size):
        comb = combs.pop(0)
        for i in range(len(comb) + 1):
            tmp = comb.copy()
            tmp.insert(i, pointer)
            combs.append(tmp) 
    return combs
    
permute(arr)

[[1, 2, 3], [2, 1, 3], [2, 3, 1], [1, 3, 2], [3, 1, 2], [3, 2, 1]]